In [ ]:
suppressPackageStartupMessages(if(!require(itsmr)) install.packages("itsmr", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(astsa)) install.packages("astsa", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(TSA)) install.packages("TSA", repos = "http://cran.us.r-project.org"))
#suppressPackageStartupMessages(if(!require(urca)) install.packages("urca", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(fracdiff)) install.packages("fracdiff", repos = "http://cran.us.r-project.org"))

suppressPackageStartupMessages(if(!require(tsoutliers)) install.packages("tsoutliers", repos = "http://cran.us.r-project.org"))

suppressPackageStartupMessages(if(!require(lmtest)) install.packages("lmtest", repos = "http://cran.us.r-project.org"))
#suppressPackageStartupMessages(if(!require(car)) install.packages("car", repos = "http://cran.us.r-project.org"))

suppressPackageStartupMessages(if(!require(forecast)) install.packages("forecast", repos = "http://cran.us.r-project.org"))
#suppressPackageStartupMessages(if(!require(tsfknn)) install.packages("tsfknn", repos = "http://cran.us.r-project.org"))

In [ ]:
# Aumentar o tamanho das figuras
options(repr.plot.width=16, repr.plot.height=12)

## Usando o pacote tsoutliers

In [ ]:
library(help="tsoutliers")

Procedimento para detecção automática descrito por [Chen & Liu (1993)](https://www.jstor.org/stable/pdf/2290724.pdf). A representação ARIMA contaminado pode ser escrito como
$$y_t=\sum_{j=1}^m\omega_j\nu_j(B)\mathbb{I}_t(T)+\frac{\theta(B)}{\varphi(B)}Z_t, \tag1$$

onde $omega_j$ representa o impacto da perturbação, $\mathbb{I}_t(T)$ representa a função de impulso e considera o valor $1$ quando $t=T$ e $0$ em caso contrário.


I. **Identificação dos *outliers***: Com um modelo ARIMA ajustado aos dados, os *outliers* são detectados e localizados verificando a significância de todos os tipos de outliers;

II. **Removendo os *outliers***: Dado um conjunto de possíveis outliers, um modelo ARIMA é escolhido e ajustado de acordo com a Eq. 1. A significância dos dados atípicos é reavaliada no novo modelo ajustado. Se for usado um procedimento de seleção ARIMA, um novo modelo poderá ser selecionado nesse estágio. **Os outliers que não são significativos são removidos do conjunto de possíveis outliers**.

## Aditivo

In [ ]:
n<-1000
tc <- rep(0, n)
tc[n*0.5] <- 1
ao <- filter(tc, filter = 0, method = "recursive")
plot(ao, main = "Outlier Aditivo - TC delta = 0", type = "l")

### Mudança transitória

\begin{equation}
\begin{aligned}
f_t(T) = \dfrac{\omega B}{1 – \delta B} \mathbb{I}_{t}(T)
\end{aligned}
\end{equation}

In [ ]:
tc_0_4 <- filter(tc, filter = 0.4, method = "recursive")
tc_0_8 <- filter(tc, filter = 0.8, method = "recursive")
plot(tc_0_4[400:600], main = "TC delta = 0.4", type = "l") # Intervalo [400,600]
plot(tc_0_8[400:600], main = "TC delta = 0.8", type = "l")

### Mudança permanente

In [ ]:
tc <- rep(0, n)
tc[n*0.5] <- 1
ls <- filter(tc, filter = 1, method = "recursive")
plot(ls, main = "Level Shift (Mudança de nivel) - TC delta = 1", type = "l")

## Dados artificiais

In [ ]:
set.seed(123)
n<-120
y <- arima.sim(model = list(ar = 0.7, ma = -0.4), n = 120)
y[15] <- -4
y[45] <- 5
y[80:120] <- y[80:120] + 5
y <- round(y, 2)

In [ ]:
itsmr::plotc(y)

In [ ]:
fit <- forecast::auto.arima(x = y, allowdrift = FALSE, ic = "bic")
pars <- tsoutliers::coefs2poly(fit)
resid <- residuals(fit)
n <- length(resid)
sigma <- 1.483 * quantile(abs(resid - quantile(resid, probs = 0.5)), probs = 0.5)

In [ ]:
lmtest::coeftest(fit)

In [ ]:
outliers_excess_ts <- tsoutliers::tso(y, types = c("AO","TC", "LS", "IO"), cval = 3., tsmethod = "arima", args.tsmethod=(list(order = c(3, 1, 1), seasonal
= list(order = c(0, 0, 0)))))
outliers_excess_ts

In [ ]:
plot(outliers_excess_ts)

In [ ]:
# Dados de vazão do rio Nilo

resNile1 <- tsoutliers::tso(y = Nile, types = c("AO", "LS", "TC"), tsmethod = "auto.arima", args.tsmethod = list(allowdrift = FALSE, ic = "bic"))
resNile1$fit$call$xreg<-NULL
resNile1

In [ ]:
plot(resNile1)

In [ ]:
# Dados de passageiros
resAirP <- tsoutliers::tso(y = log(AirPassengers), types = c("AO", "LS", "TC"), maxit = 1, discard.method = "bottom-up", tsmethod = "arima",
args.tsmethod = list(order = c(0, 1, 1), seasonal = list(order = c(0, 1, 1))))
resAirP

In [ ]:
plot(resAirP)

In [ ]:
data("bde9915")
gipi <- log(bde9915$gipi)
ce <- calendar.effects(gipi)
resGIPI1 <- tsoutliers::tso(y = gipi, xreg = ce, cval = 3.5, types = c("AO", "LS", "TC", "SLS"), maxit = 1, discard.method = "bottom-up",
tsmethod = "arima", args.tsmethod = list(order = c(0, 1, 1), seasonal = list(order = c(0, 1, 1))))
resGIPI1

In [ ]:
plot(resGIPI1)

In [ ]:
resGIPI2 <- tsoutliers::tso(y = gipi, xreg = ce, types = c("AO", "LS", "TC", "SLS"), maxit = 1, discard.method = "bottom-up", tsmethod = "auto.arima",
args.tsmethod = list(allowdrift = FALSE, ic = "bic"))
resGIPI2

In [ ]:
plot(resGIPI2)

In [ ]:
resGIPI3 <- tso(y = gipi, xreg = ce, cval = 3.5, types = c("AO", "LS", "TC", "SLS"), maxit = 1, discard.method = "en-masse",
tsmethod = "arima", args.tsmethod = list(order = c(0, 1, 1),
seasonal = list(order = c(0, 1, 1))))
resGIPI3

In [ ]:
plot(resGIPI3)

In [ ]:
url <- "https://bit.ly/47rGgfe"
abhutondot <- read.csv(url, header=TRUE)

boys_ts <- ts(abhutondot$boys, frequency=1, start = abhutondot$year[1])
girls_ts <- ts(abhutondot$girls, frequency=1, start = abhutondot$year[1])

delta_ts <- boys_ts - girls_ts
excess_ts <- delta_ts/girls_ts
plot(excess_ts)

In [ ]:
outliers_excess_ts <- tso(excess_ts, types = c("TC", "AO", "LS", "IO", "SLS"))
outliers_excess_ts

In [ ]:
plot(outliers_excess_ts)

In [ ]:
outliers_excess_ts$outliers

In [ ]:
# Tamanho de amostra
n <- length(excess_ts)
outliers_idx <- outliers_excess_ts$outliers$ind
mo_tc <- outliers("TC", outliers_idx)
tc <- outliers.effects(mo_tc, n)

In [ ]:
# converting to a number
coefhat <- as.numeric(outliers_excess_ts$outliers["coefhat"])

# obtaining the transient change data with same magnitude as determined by the tso() function
tc_effect <- coefhat*tc

# definining a time series for the transient change data
tc_effect_ts <- ts(tc_effect, frequency = frequency(excess_ts), start = start(excess_ts))

# subtracting the transient change intervention to the original time series, obtaining the pre-intervention time series
excess_wo_ts <- excess_ts - tc_effect_ts

# plot of the original, the pre-intervention and transient change time series
plot(cbind(excess_ts, excess_wo_ts, tc_effect_ts))

In [ ]:
plot(excess_ts, type ='l', col='blue')
lines(excess_wo_ts, col = 'red', lty = 3, type ='l')

In [ ]:
sarima(excess_wo_ts, p=0, d=0, q=0)